<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

The following example is a 1-D minimization problem, with many
local minima superimposed on a parabola.


In [ ]:
import numpy as np
from scipy.optimize import basinhopping
func = lambda x: np.cos(14.5 * x - 0.3) + (x + 0.2) * x
x0 = [1.]

Basinhopping, internally, uses a local minimization algorithm. We will use
the parameter `minimizer_kwargs` to tell basinhopping which algorithm to
use and how to set up that minimizer. This parameter will be passed to
`scipy.optimize.minimize`.


In [ ]:
minimizer_kwargs = {"method": "BFGS"}
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=200)
print("global minimum: x = %.4f, f(x) = %.4f" % (ret.x, ret.fun))

global minimum: x = -0.1951, f(x) = -1.0009

Next consider a 2-D minimization problem. Also, this time, we
will use gradient information to significantly speed up the search.


In [ ]:
def func2d(x):
    f = np.cos(14.5 * x[0] - 0.3) + (x[1] + 0.2) * x[1] + (x[0] +
                                                           0.2) * x[0]
    df = np.zeros(2)
    df[0] = -14.5 * np.sin(14.5 * x[0] - 0.3) + 2. * x[0] + 0.2
    df[1] = 2. * x[1] + 0.2
    return f, df

We'll also use a different local minimization algorithm. Also, we must tell
the minimizer that our function returns both energy and gradient (Jacobian).


In [ ]:
minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}
x0 = [1.0, 1.0]
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=200)
print("global minimum: x = [%.4f, %.4f], f(x) = %.4f" % (ret.x[0],
                                                          ret.x[1],
                                                          ret.fun))

global minimum: x = [-0.1951, -0.1000], f(x) = -1.0109

Here is an example using a custom step-taking routine. Imagine you want
the first coordinate to take larger steps than the rest of the coordinates.
This can be implemented like so:


In [ ]:
class MyTakeStep:
   def __init__(self, stepsize=0.5):
       self.stepsize = stepsize
       self.rng = np.random.default_rng()
   def __call__(self, x):
       s = self.stepsize
       x[0] += self.rng.uniform(-2.*s, 2.*s)
       x[1:] += self.rng.uniform(-s, s, x[1:].shape)
       return x

Since ``MyTakeStep.stepsize`` exists basinhopping will adjust the magnitude
of `stepsize` to optimize the search. We'll use the same 2-D function as
before


In [ ]:
mytakestep = MyTakeStep()
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=200, take_step=mytakestep)
print("global minimum: x = [%.4f, %.4f], f(x) = %.4f" % (ret.x[0],
                                                          ret.x[1],
                                                          ret.fun))

global minimum: x = [-0.1951, -0.1000], f(x) = -1.0109

Now, let's do an example using a custom callback function which prints the
value of every minimum found


In [ ]:
def print_fun(x, f, accepted):
        print("at minimum %.4f accepted %d" % (f, int(accepted)))

We'll run it for only 10 basinhopping steps this time.


In [ ]:
rng = np.random.default_rng()
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=10, callback=print_fun, seed=rng)

at minimum 0.4159 accepted 1
at minimum -0.4317 accepted 1
at minimum -1.0109 accepted 1
at minimum -0.9073 accepted 1
at minimum -0.4317 accepted 0
at minimum -0.1021 accepted 1
at minimum -0.7425 accepted 1
at minimum -0.9073 accepted 1
at minimum -0.4317 accepted 0
at minimum -0.7425 accepted 1
at minimum -0.9073 accepted 1

The minimum at -1.0109 is actually the global minimum, found already on the
8th iteration.